<a href="https://colab.research.google.com/github/k4ilham/collab/blob/main/AnalisisSentimenCapres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [187]:
url = 'https://raw.githubusercontent.com/k4ilham/dataset-colab/main/sepakbola-indonesia/dataset.csv'
df = pd.read_csv(url,encoding='latin1')
df = df[['userName','text','date']]
df = df.sort_values(by='date',ascending=False)
df.head()

,userName,text,date
48,pamungkas_bie,tapi sok kali,2024/05/06 09:48:58
47,arifuwik533gmail.com3,"Meskipun sampai di juara empat,kalian tetap ju...",2024/05/05 12:11:24
46,rendangque,Pak @erickthohir @shintaeyong7777 bawa pemain ...,2024/05/04 14:50:07
45,rfkyrivldo_,"Masih ada 1 lagi , lwn squad chetahh????",2024/05/04 11:34:10
44,raflii_fajrr,@rfkyrivldo_ apo nan assalamu'alaikum lah kala...,2024/05/04 11:01:41


In [188]:
print(df.columns)

Index(['userName', 'text', 'date'], dtype='object')


In [189]:
df = df[['text','userName','date']]
df

,text,userName,date
48,tapi sok kali,pamungkas_bie,2024/05/06 09:48:58
47,"Meskipun sampai di juara empat,kalian tetap ju...",arifuwik533gmail.com3,2024/05/05 12:11:24
46,Pak @erickthohir @shintaeyong7777 bawa pemain ...,rendangque,2024/05/04 14:50:07
45,"Masih ada 1 lagi , lwn squad chetahh????",rfkyrivldo_,2024/05/04 11:34:10
44,@rfkyrivldo_ apo nan assalamu'alaikum lah kala...,raflii_fajrr,2024/05/04 11:01:41
...,...,...,...
743,??????????????????????????????????????????????...,alviaass,2024/04/26 13:39:30
742,Waalaikumsalam sidang Jumat yang berbahagia ??,aku___fahmi___,2024/04/26 13:38:46
741,Alhamdulillah ?? nggak sia* nobar sampai jelan...,hannnnn_099,2024/04/26 13:38:45
740,"Muannnnnntep, Budakssss????",ikystywn_,2024/04/26 13:37:18


#Cleaning Data

##View the number of rows and columns

In [190]:
df.shape

(789, 3)

##Delete duplicate data

In [191]:
df.duplicated().sum()

97

In [192]:
df = df.drop_duplicates(subset=['text'])

In [193]:
df.duplicated().sum()

0

##Remove empty values

In [194]:
df.isnull().sum()

text        1
userName    0
date        0
dtype: int64

In [195]:
df = df.dropna()

In [196]:
df.isnull().sum()

text        0
userName    0
date        0
dtype: int64

In [197]:
df.shape

(563, 3)

## Data cleaning process

In [198]:
df

,text,userName,date
48,tapi sok kali,pamungkas_bie,2024/05/06 09:48:58
47,"Meskipun sampai di juara empat,kalian tetap ju...",arifuwik533gmail.com3,2024/05/05 12:11:24
46,Pak @erickthohir @shintaeyong7777 bawa pemain ...,rendangque,2024/05/04 14:50:07
45,"Masih ada 1 lagi , lwn squad chetahh????",rfkyrivldo_,2024/05/04 11:34:10
44,@rfkyrivldo_ apo nan assalamu'alaikum lah kala...,raflii_fajrr,2024/05/04 11:01:41
...,...,...,...
743,??????????????????????????????????????????????...,alviaass,2024/04/26 13:39:30
742,Waalaikumsalam sidang Jumat yang berbahagia ??,aku___fahmi___,2024/04/26 13:38:46
741,Alhamdulillah ?? nggak sia* nobar sampai jelan...,hannnnn_099,2024/04/26 13:38:45
740,"Muannnnnntep, Budakssss????",ikystywn_,2024/04/26 13:37:18


## Convert words to lowercase


In [199]:
df['text'] = df['text'].str.lower()  # case folding

# Tokenizing and cleaning
df['text'] = df['text'].str.replace(r'https\S+', ' ', regex=True)  # remove https
df['text'] = df['text'].str.replace(r'@\S+', ' ', regex=True)  # remove mentions
df['text'] = df['text'].str.replace(r'#\S+', ' ', regex=True)  # remove hashtags
df['text'] = df['text'].str.replace(r"\'\w+", ' ', regex=True)  # remove next character
df['text'] = df['text'].str.replace(r'[^\w\s]', ' ', regex=True)  # remove punctuation
df['text'] = df['text'].str.replace(r'\w*\d+\w*', ' ', regex=True)  # remove numbers
df['text'] = df['text'].str.replace(r'\s+', ' ', regex=True)  # remove excessive whitespace


In [200]:
def clean_twitter_text(text):
  # Menghapus mention (@username)
  text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Menambahkan + untuk menghapus seluruh username, bukan hanya karakter pertama

  # Menghapus hashtag (#hashtag)
  text = re.sub(r'#\w+', '', text)

  # Menghapus retweet (RT diikuti spasi)
  text = re.sub(r'RT[\s]+', '', text)

  # Menghapus URL (http atau https diikuti karakter non-spasi)
  text = re.sub(r'https?://\S+', '', text)

  # Menghapus karakter non-alfanumerik kecuali spasi
  text = re.sub(r'[^A-Za-z0-9 ]', '', text)

  # Menghapus spasi tambahan di dalam teks (mengganti beberapa spasi dengan satu spasi)
  text = re.sub(r'\s+', ' ', text).strip()  # Menggunakan spasi tunggal di sini dan menambahkan strip() untuk menghapus spasi di awal/akhir

  return text


In [201]:
df['text'] = df['text'].apply(clean_twitter_text)

In [202]:
df = df[df['text'].str.strip() != '']

In [203]:
df

,text,userName,date
48,tapi sok kali,pamungkas_bie,2024/05/06 09:48:58
47,meskipun sampai di juara empat kalian tetap ju...,arifuwik533gmail.com3,2024/05/05 12:11:24
46,pak bawa pemain timnas ke bali pak suruh merek...,rendangque,2024/05/04 14:50:07
45,masih ada lagi lwn squad chetahh,rfkyrivldo_,2024/05/04 11:34:10
44,apo nan assalamu lah kalah alah,raflii_fajrr,2024/05/04 11:01:41
...,...,...,...
744,otw olimpiade,alifianfauzie,2024/04/26 13:40:57
742,waalaikumsalam sidang jumat yang berbahagia,aku___fahmi___,2024/04/26 13:38:46
741,alhamdulillah nggak sia nobar sampai jelang su...,hannnnn_099,2024/04/26 13:38:45
740,muannnnnntep budakssss,ikystywn_,2024/04/26 13:37:18


#Preprocesing

##Normalisasi

In [204]:
norm = {" yg ": " yang ", " nggak ": " tidak ", " bangetdari ": " banget dari ", " vibes ": " suasana ", " mantab ": " keren "}

In [205]:
def normalisasi(str_text):
  for i in norm:
    str_text = str_text.replace(i, norm[i])
  return str_text

In [206]:
df['text'] = df['text'].apply(lambda x: normalisasi(x))

<ipython-input-206-620c41629945>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: normalisasi(x))


In [207]:
df

,text,userName,date
48,tapi sok kali,pamungkas_bie,2024/05/06 09:48:58
47,meskipun sampai di juara empat kalian tetap ju...,arifuwik533gmail.com3,2024/05/05 12:11:24
46,pak bawa pemain timnas ke bali pak suruh merek...,rendangque,2024/05/04 14:50:07
45,masih ada lagi lwn squad chetahh,rfkyrivldo_,2024/05/04 11:34:10
44,apo nan assalamu lah kalah alah,raflii_fajrr,2024/05/04 11:01:41
...,...,...,...
744,otw olimpiade,alifianfauzie,2024/04/26 13:40:57
742,waalaikumsalam sidang jumat yang berbahagia,aku___fahmi___,2024/04/26 13:38:46
741,alhamdulillah tidak sia nobar sampai jelang su...,hannnnn_099,2024/04/26 13:38:45
740,muannnnnntep budakssss,ikystywn_,2024/04/26 13:37:18


##Stopword

In [208]:
!pip install Sastrawi

In [209]:
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
more_stop_words = ["tidak"]

stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text

In [210]:
df['text'] = df['text'].apply(lambda x: stopword(x))

<ipython-input-210-6bc94d329983>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: stopword(x))


In [211]:
df.head()

,text,userName,date
48,sok kali,pamungkas_bie,2024/05/06 09:48:58
47,meskipun di juara empat kalian tetap juara sat...,arifuwik533gmail.com3,2024/05/05 12:11:24
46,pak bawa pemain timnas bali pak suruh latihan ...,rendangque,2024/05/04 14:50:07
45,ada lwn squad chetahh,rfkyrivldo_,2024/05/04 11:34:10
44,apo nan assalamu lah kalah alah,raflii_fajrr,2024/05/04 11:01:41


##Tokenize

In [212]:
tokenized = df['text'].apply(lambda x:x.split())


In [213]:
tokenized

48                                           [sok, kali]
47     [meskipun, di, juara, empat, kalian, tetap, ju...
46     [pak, bawa, pemain, timnas, bali, pak, suruh, ...
45                            [ada, lwn, squad, chetahh]
44                [apo, nan, assalamu, lah, kalah, alah]
                             ...                        
744                                     [otw, olimpiade]
742          [waalaikumsalam, sidang, jumat, berbahagia]
741    [alhamdulillah, sia, nobar, jelang, subuh, rel...
740                            [muannnnnntep, budakssss]
739                                            [menyala]
Name: text, Length: 516, dtype: object

##Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(text_cleaning):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []

  for w in text_cleaning:
    dt = stemmer.stem(w)
    do.append(dt)

  d_clean = []
  d_clean = " ".join(do)
  print(d_clean)
  return d_clean

tokenized = tokenized.apply(stemming)

sok kali
meski di juara empat kalian tetap juara satu indonesia semangat terus garuda kamu hebat
pak bawa main timnas bal pak suruh latih passing jugling pakai bola api yang pernah laku dulu dia kenal akan buat ajar terlalu lama bawa bola
ada lwn squad chetahh
apo nan assalamu lah kalah alah
air alir dalam goa lihat orang jalan balik bantu doa moga timnas hasil baik
rafael ganteng bgttttttt
ayo timnas menang masuk olimpiade paris
indonesia iraq
terimankasih pak erik thohir smua sudah terliba atas keeberhhasilan timnas jauh
semangat timnas indonesia nyala timnas indonesia
i proud of timnas indonesia
nyala abang
bismillahirrahmanirrahim
bangga
terima kasih timnas juang keras tetap semangat jangan serah proud hormat
tetep cinta tinnas apa hasil kalian sdh juang tks my hero
apa hasil malam kami ttep bangga juang kalian trimakasih jauh semangat garuda
tetap main jujur walopun kaaj wasit kalian lah juara semangat
wasit korsel vs indo piye jek enak jaman ku
padahal banyak main gk main kan sam

##Save to csv

In [ ]:
tokenized.to_csv("hasil_reprocessing.csv", index=False)

buat terjemahan secara manual di google sheet